In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [14]:
class NeuralNetwork(nn.Module):  # building the structure of the neural network
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [15]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_prob = nn.Softmax(dim=1)(logits)  # passing logits, values of [-infty, infty] through softmax gives prediction probabilities
y_pred = pred_prob.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([2])


In [16]:
input_image = torch.rand(3,28,28)
print(input_image.size())
flatten = nn.Flatten()  # flatten converts 2D images to a contiguous array of 1D pixels
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 28, 28])
torch.Size([3, 784])


In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.4155, -0.5589,  0.2473,  0.7436, -0.1416, -0.2666,  0.2172, -0.1430,
          0.0735,  0.2477,  0.2243,  0.0604, -0.2217, -0.4496,  0.2374,  0.1443,
         -0.5686,  0.1940, -0.3808, -0.2545],
        [-0.3997, -0.5738,  0.0823,  0.7159,  0.0739, -0.3660,  0.4281, -0.2775,
          0.3078,  0.0474,  0.0374,  0.0119, -0.3232, -0.2531,  0.0965,  0.4929,
         -0.0331,  0.0199, -0.5695, -0.3444],
        [ 0.0408, -0.4701,  0.4167,  0.8880, -0.0043, -0.4509,  0.0025, -0.2025,
          0.3272,  0.1951, -0.4483, -0.0354, -0.0908, -0.0114,  0.0630,  0.0481,
         -0.5494, -0.0236, -0.3765, -0.1958]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.2473, 0.7436, 0.0000, 0.0000, 0.2172, 0.0000, 0.0735,
         0.2477, 0.2243, 0.0604, 0.0000, 0.0000, 0.2374, 0.1443, 0.0000, 0.1940,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0823, 0.7159, 0.0739, 0.0000, 0.4281, 0.0000, 0.3078,
         0.0474, 0.0374, 0.0119, 0.0000, 0.0000, 0.09

In [12]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)  # logits are raw values in the range [-infty, infty]
softmax = nn.Softmax(dim=1)
pred_prob = softmax(logits)

In [13]:
print(f"Model structure: {model}\n\n")
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size :{param.size()} | Values: {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size :torch.Size([512, 784]) | Values: tensor([[-0.0143, -0.0304, -0.0207,  ...,  0.0042,  0.0166,  0.0017],
        [ 0.0136, -0.0252, -0.0243,  ..., -0.0210, -0.0137,  0.0102]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size :torch.Size([512]) | Values: tensor([ 0.0278, -0.0152], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size :torch.Size([512, 512]) | Values: tensor([[-0.0233, -0.0046, -0.0331,  ...,  0.0302,  0.0365, -0.0162],
        [ 0.0275,  0.0396, -0.0093,  ...,  0.0395,  0.0391,  0.0240]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Siz